ブログ

In [ ]:
from pathlib import Path
import geopandas as gpd
import pandas as pd
import numpy as np
from shapely.geometry import Polygon
import matplotlib.pyplot as plt
import japanize_matplotlib
from tqdm import tqdm

In [ ]:
# データ読み込み
files = list(Path('./csv').glob('*.csv'))
print(files[0])

In [ ]:
df = pd.DataFrame()
for filepath in files:
    tmp = pd.read_csv(filepath, dtype=str)
    df = pd.concat([df, tmp]).reset_index(drop=True)

In [ ]:
# ポリゴンに変換する
df['lod0RoofEdge'] = df['lod0RoofEdge'].apply(lambda row: row.split())

In [ ]:
df['lod0RoofEdge'] = df['lod0RoofEdge'].apply(lambda row: np.reshape(row, (-1,3)).astype(float))

In [ ]:
df['lod0RoofEdge'] = df['lod0RoofEdge'].apply(Polygon)

In [ ]:
gdf = gpd.GeoDataFrame(df)
gdf.set_geometry('lod0RoofEdge', crs='EPSG:6697', inplace=True)

In [ ]:
gdf.shape

---
ビルのboundaryをとり、角度と距離を計算する

In [ ]:
#　角度と距離を計算する
from typing import List
def calc_angle_dist(elem: Polygon) -> List[float, float]:
    b = np.array(elem.minimum_rotated_rectangle.boundary.coords)
    vec = [b[i] - b[i+1] for i in range(4)]
    return [[np.arctan2(*v), np.linalg.norm(v)] for v in vec]

In [ ]:
print(calc_angle_dist(gdf['lod0RoofEdge'][0]))

In [ ]:
tqdm.pandas()

In [ ]:
gdf['boundary'] = gdf['lod0RoofEdge'].progress_apply(calc_angle_dist)

In [ ]:
gdf_explode = gdf[['boundary', 'city_name']].explode('boundary').reset_index(drop=True)

In [ ]:
gdf_explode['angle'] = gdf_explode['boundary'].str[0]
gdf_explode['dist'] = gdf_explode['boundary'].str[1]

In [ ]:
# save point
# gdf_explode[['city_name','angle','dist']].to_csv('data.csv',index=False)
# gdf_explode = pd.read_csv('data.csv')

In [ ]:
gdf_explode.query('city_name=="東京都墨田区"')['angle'].apply([min,max])

In [ ]:
gdf_explode.loc[gdf_explode['angle']>np.pi, 'angle'] -= np.pi #.apply([min,max])

In [ ]:
gdf_explode.shape

In [ ]:
gdf_explode.head()

In [ ]:
gdf_explode['city_name'].nunique()

In [ ]:
gdf_explode['city_name'].unique()

どれだけ値が集中しているかでsortする

In [ ]:
result = pd.DataFrame()
for i, city in enumerate(gdf_explode.city_name.unique()):
    tmpdf = gdf_explode.query('city_name==@city')
    val = pd.cut(tmpdf['angle'], 40).value_counts(normalize=True).reset_index(drop=True).rename(city)
    result = pd.concat([result, val],axis=1)

In [ ]:
cities = (result.T[0] + result.T[2] + result.T[4]).sort_values(ascending=False).index
cities

全部の区をplotする

In [ ]:
N = 40
bottom = 1
width = (2*np.pi) / N
max_height = 4
theta = np.linspace(0.0, 2 * np.pi, N, endpoint=False)

In [ ]:
fig, axes = plt.subplots(6,4,figsize=(20,35), subplot_kw={'projection': 'polar'})
for i,city in enumerate(cities):
    # prepare data
    tmpdf = gdf_explode.query('city_name==@city')
    # radii, _ ,_ = plt.hist(tmpdf['angle'], weights=tmpdf['dist'], bins=N)
    radii, _ = np.histogram(tmpdf['angle'], weights=tmpdf['dist'], bins=N)
    radii = radii / max(radii) * max_height
    
    # plot
    ax = axes[i//4][i%4]
    ax.set_title(city)
    bars = ax.bar(theta, radii, width=width, bottom=bottom)

    # x-labels setting
    ax.set_xlim([-np.pi, np.pi])
    ax.set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
    ax.set_xticklabels(["SW", "W", "NW", "N", "NE", "E", "SE", "S",])
    ax.set_theta_direction(-1)
    ax.set_theta_zero_location("N")

    # y-label setting
    ax.set_yticklabels([])

    # Use custom colors and opacity
    for r, bar in zip(radii, bars):
        bar.set_facecolor(plt.cm.jet(r / 10.))
        bar.set_alpha(0.8)
plt.show()

中央区だけプロットする

In [ ]:
fig, ax = plt.subplots(1,1,figsize=(7,7), subplot_kw={'projection': 'polar'})
# prepare data
city = '東京都中央区'
tmpdf = gdf_explode.query('city_name==@city')
# radii, _ ,_ = plt.hist(tmpdf['angle'], weights=tmpdf['dist'], bins=N)
radii, _ = np.histogram(tmpdf['angle'], weights=tmpdf['dist'], bins=N)
radii = radii / max(radii) * max_height

# plot
ax.set_title(city)
bars = ax.bar(theta, radii, width=width, bottom=bottom)

# x-labels setting
ax.set_xlim([-np.pi, np.pi])
ax.set_xticks(np.linspace(-np.pi, np.pi, 9)[1:])
ax.set_xticklabels(["SW", "W", "NW", "N", "NE", "E", "SE", "S",])
ax.set_theta_direction(-1)
ax.set_theta_zero_location("N")

# y-label setting
ax.set_yticklabels([])

# Use custom colors and opacity
for r, bar in zip(radii, bars):
    bar.set_facecolor(plt.cm.jet(r / 10.))
    bar.set_alpha(0.8)
plt.show()